In [1]:
!python3 -m vllm.entrypoints.api_server --host "0.0.0.0" --model mmnga/Xwin-LM-7B-AWQ-calib-ja-100k --quantization awq

INFO 09-26 01:28:13 llm_engine.py:72] Initializing an LLM engine with config: model='mmnga/Xwin-LM-7B-AWQ-calib-ja-100k', tokenizer='mmnga/Xwin-LM-7B-AWQ-calib-ja-100k', tokenizer_mode=auto, revision=None, trust_remote_code=False, dtype=torch.float16, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)
INFO 09-26 01:31:33 llm_engine.py:202] # GPU blocks: 2118, # CPU blocks: 512
INFO:     Started server process [114]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO 09-26 01:32:39 async_llm_engine.py:371] Received request c3d74a915af34f4880eb6516c9ac54df: prompt: 'JSのフレームワークのRemixについて知っていることを教えて。', sampling params: SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, temperature=0.3, top_p=1.0, top_k=-1, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], ignore_eos=False, max_tokens=512, logprobs=No

In [8]:
!pip install autoawq==0.1.0 transformers

  Obtaining dependency information for autoawq from https://files.pythonhosted.org/packages/e0/45/4ec85c794a9436f14b8ed408511300622f6fc4ca8ab58ec7e34ddf5f2496/autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl.metadata
  Using cached autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl.metadata (12 kB)
Using cached autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl (17.4 MB)


In [1]:
%%time
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "mmnga/Xwin-LM-7B-AWQ-calib-ja-100k"

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)

def chat(prompt, max_new_tokens):
    tokens = tokenizer(
        prompt,
        return_tensors='pt'
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=max_new_tokens
    )
    return tokenizer.decode(generation_output[0])

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Replacing layers...: 100% 32/32 [00:02<00:00, 10.91it/s]


In [7]:
%%time
prompt = """日本の観光地の平均気温をjsonで書き出すと、"""
print(chat(prompt, 256))

<s> 日本の観光地の平均気温をjsonで書き出すと、以下のような形式でなければならない。
```json
[
  {
    "名称": "浜名湖",
    "平均気温": 13.2
  },
  {
    "名称": "富嶽百景",
    "平均気温": 11.6
  },
  // ...
]
```
このJSON形式で、各観光地の名称と平均気温を格納しておくことで、JavaScriptやTypeScript、またはPythonなどのプログラミング言語を使って、観光地の平均気温を取得し、表示することができます。

## 観光地の平均気温を取得する

以下に、Pythonで観光地の平均気温を取得する例を示し
CPU times: user 17.6 s, sys: 9.25 ms, total: 17.6 s
Wall time: 17.6 s


In [8]:
%%time
prompt = """小学生向けに計算問題(2桁の四則演算)を5問出力してください。まず問題を並べて、そのあとに解答のみを並べてください。

"""
print(chat(prompt, 256))

<s> 小学生向けに計算問題(2桁の四則演算)を5問出力してください。まず問題を並べて、そのあとに解答のみを並べてください。

1. 4 + 6 = ?
2. 9 - 3 = ?
3. 12 - 5 = ?
4. 8 + 1 = ?
5. 14 - 7 = ?</s>
CPU times: user 2.73 s, sys: 11.9 ms, total: 2.74 s
Wall time: 2.74 s
